# Configuration

In [5]:
!pip install pydicom
!pip install optuna
!pip install scikit-image


[notice] A new release of pip available: 22.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


     --------------------------------------- 12.8/12.8 MB 10.1 MB/s eta 0:00:00
     ---------------------------------------- 2.6/2.6 MB 8.8 MB/s eta 0:00:00
     ------------------------------------- 227.6/227.6 kB 13.6 MB/s eta 0:00:00
     ------------------------------------- 315.8/315.8 kB 19.1 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: Pillow 10.0.1
    Uninstalling Pillow-10.0.1:
      Successfully uninstalled Pillow-10.0.1


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'E:\\00MY WORKSPACE\\my_venv\\Lib\\site-packages\\~il\\_imaging.cp310-win_amd64.pyd'
Check the permissions.


[notice] A new release of pip available: 22.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
pip install torch==1.0.2

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement torch==1.0.2 (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0, 2.0.1, 2.1.0, 2.1.1, 2.1.2, 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.5.0, 2.5.1, 2.6.0)
ERROR: No matching distribution found for torch==1.0.2

[notice] A new release of pip available: 22.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
pip install --upgrade Pillow

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import pydicom
import os
import numpy as np
import matplotlib.pyplot as plt
#from skimage.transform import resize
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import optuna

from google.colab import drive

# Monter Google Drive avec force remount
#drive.mount('/content/drive', force_remount=True)


ModuleNotFoundError: No module named 'torch'

# Chargement du dataset

In [3]:


def load_dicom_series(directory):
    """
    Charge une série d'images DICOM à partir d'un dossier.
    """
    dicom_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.dcm')]
    dicom_files.sort()  # Assurez-vous que les fichiers sont dans le bon ordre
    slices = [pydicom.dcmread(f) for f in dicom_files]
    return slices

def get_pixel_array(dicom_slices):
    """
    Convertit une série de slices DICOM en un tableau numpy 3D.
    """
    volume = np.stack([s.pixel_array for s in dicom_slices], axis=0)
    return volume

# Exemple d'utilisation
#dicom_directory = "/content/drive/MyDrive/Colab/ganglions_detection/dataset"
dicom_directory = "/dataset"
dicom_slices = load_dicom_series(dicom_directory)
volume = get_pixel_array(dicom_slices)
print("Volume shape:", volume.shape)

NotADirectoryError: [WinError 267] The directory name is invalid: '/dataset'

# Prétraitement

In [ ]:

def normalize_volume(volume):
    """
    Normalise les intensités des pixels entre 0 et 1.
    """
    volume = (volume - np.min(volume)) / (np.max(volume) - np.min(volume))
    return volume

def resize_volume(volume, target_shape):
    """
    Redimensionne le volume à une taille cible.
    """
    resized_volume = np.zeros(target_shape)
    for i in range(volume.shape[0]):
        resized_volume[i] = resize(volume[i], target_shape[1:], preserve_range=True)
    return resized_volume

# Exemple de prétraitement
volume_normalized = normalize_volume(volume)
target_shape = (volume.shape[0], 256, 256)  # Redimensionner à 256x256
volume_resized = resize_volume(volume_normalized, target_shape)
print("Volume resized shape:", volume_resized.shape)

# Modèle

## Création du modèle

In [ ]:


class UNet(nn.Module):
    def __init__(self, in_channels=1, out_channels=1):
        super(UNet, self).__init__()
        # Encoder
        self.encoder1 = self.conv_block(in_channels, 64)
        self.encoder2 = self.conv_block(64, 128)
        self.encoder3 = self.conv_block(128, 256)
        # Decoder
        self.decoder1 = self.conv_block(256, 128)
        self.decoder2 = self.conv_block(128, 64)
        self.final_layer = nn.Conv2d(64, out_channels, kernel_size=1)
        # Classification
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(256, 1),  # Classification binaire
            nn.Sigmoid()
        )

    def conv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        # Encoder
        x1 = self.encoder1(x)
        x2 = self.encoder2(F.max_pool2d(x1, 2))
        x3 = self.encoder3(F.max_pool2d(x2, 2))
        # Decoder
        x = F.interpolate(x3, scale_factor=2, mode='bilinear', align_corners=True)
        x = self.decoder1(x + x2)
        x = F.interpolate(x, scale_factor=2, mode='bilinear', align_corners=True)
        x = self.decoder2(x + x1)
        segmentation = self.final_layer(x)
        # Classification
        classification = self.classifier(x3)
        return segmentation, classification

# Exemple d'initialisation du modèle
model = UNet(in_channels=1, out_channels=1)
print(model)

In [ ]:
dummy_input = torch.rand((1, 1, 128, 128, 128)).to(device)
output = model(dummy_input)
print("Output Shape:", output.shape)  # Devrait être (1, 1, 128, 128, 128)


## Entrainement du modèle

In [3]:


# Fonction de perte
def dice_loss(pred, target):
    smooth = 1.
    pred_flat = pred.view(-1)
    target_flat = target.view(-1)
    intersection = (pred_flat * target_flat).sum()
    return 1 - (2. * intersection + smooth) / (pred_flat.sum() + target_flat.sum() + smooth)

# Optimiseur
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Boucle d'entraînement
for epoch in range(10):  # 10 époques
    model.train()
    for batch in dataloader:  # Remplacez par votre DataLoader
        inputs, masks, labels = batch
        optimizer.zero_grad()
        outputs, classifications = model(inputs)
        loss_seg = dice_loss(outputs, masks)
        loss_cls = F.binary_cross_entropy(classifications, labels)
        loss = loss_seg + loss_cls
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

NameError: name 'model' is not defined

# Evaluation du modèle

In [ ]:
def evaluate_model(model, val_loader, threshold=0.5):
    model.eval()
    dice_scores, iou_scores = [], []

    with torch.no_grad():
        for batch in val_loader:
            images, masks = batch['image'].to(device), batch['mask'].to(device)
            outputs = model(images)
            outputs = torch.sigmoid(outputs)  # Appliquer la sigmoid pour obtenir des probas

            for i in range(len(images)):
                dice_scores.append(dice_score(outputs[i], masks[i], threshold))
                iou_scores.append(iou_score(outputs[i], masks[i], threshold))

    print(f"⚡ Dice Score moyen : {np.mean(dice_scores):.4f}")
    print(f"⚡ IoU moyen : {np.mean(iou_scores):.4f}")

# Lancer l'évaluation
evaluate_model(model, val_loader)


#Optimisation du modèle

In [ ]:


def objective(trial):
    lr = trial.suggest_float("lr", 1e-5, 1e-3)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    # Entraînement et évaluation
    return validation_loss  # Remplacez par votre métrique de validation

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=10)
print("Meilleurs hyperparamètres:", study.best_params)

# Prédiction du modèle

In [ ]:
def predict(model, input_volume):
    model.eval()
    with torch.no_grad():
        input_tensor = torch.tensor(input_volume).unsqueeze(0).unsqueeze(0).float()
        segmentation, classification = model(input_tensor)
        return segmentation.squeeze().numpy(), classification.item()

# Exemple de prédiction
segmentation_map, classification = predict(model, volume_resized[0])
print("Classification:", "Pathologique" if classification > 0.5 else "Normal")
plt.imshow(segmentation_map, cmap='gray')
plt.show()